In [3]:
import pygame
import serial
import numpy as np
import math

# ==== Settings ====
PORT = "COM5"   # change to your Arduino port
BAUD = 115200
ROWS, COLS = 12, 12
HEIGHT_SCALE = 0.15  # spike height scaling
TILT_X = 40          # tilt grid in X
TILT_Y = 20          # tilt grid in Y
ZOOM = 25            # spacing between cells

# ==== Colors ====
BG_COLOR = (10, 10, 20)
SPIKE_COLOR = (0, 200, 255)

# ==== Init ====
pygame.init()
WIDTH, HEIGHT = 800, 300
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
ser = serial.Serial(PORT, BAUD, timeout=1)

def project_3d(x, y, z):
    """Simple 3D to 2D projection"""
    px = WIDTH // 2 + int((x - y) * TILT_X / 50)
    py = HEIGHT // 2 + int((x + y) * TILT_Y / 50) - z
    return px, py

def draw_spikes(values):
    screen.fill(BG_COLOR)

    for r in range(ROWS):
        for c in range(COLS):
            v = values[r, c]

            # Clamp input between 0 and 300
            v = max(0, min(300, v))

            # Invert value so that 0 is max height, 300 is min height
            h = int((300 - v) * HEIGHT_SCALE)

            # Base position (3D coordinates)
            x, y = c * ZOOM, r * ZOOM
            base = project_3d(x, y, 0)
            top = project_3d(x, y, h)

            # Draw spike
            pygame.draw.line(screen, SPIKE_COLOR, base, top, 3)
            pygame.draw.circle(screen, (200, 200, 200), base, 2)

    pygame.display.flip()

# ==== Main loop ====
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    line = ser.readline().decode().strip()
    if line:
        try:
            data = np.array([int(x) for x in line.split(",")])
            if len(data) == ROWS * COLS:
                values = data.reshape((ROWS, COLS))
                draw_spikes(values)
        except:
            pass

    clock.tick(30)

ser.close()
pygame.quit()


In [2]:
import pygame
import serial
import numpy as np
import math
import time

# ==== Settings ====
PORT = "COM5"   # change to your Arduino port
BAUD = 115200
ROWS, COLS = 12, 12
HEIGHT_SCALE = 0.15  # spike height scaling
TILT_X = 40          # tilt grid in X
TILT_Y = 20          # tilt grid in Y
ZOOM = 25            # spacing between cells

# ==== Colors ====
BG_COLOR = (10, 10, 20)
SPIKE_COLOR = (0, 200, 255)

# ==== Init ====
pygame.init()
pygame.display.set_caption("BiomechDance Mat — Live Pressure Map")
WIDTH, HEIGHT = 1024, 700
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
try:
    ser = serial.Serial(PORT, BAUD, timeout=1)
except Exception as e:
    ser = None
    print("Serial not available:", e)

# Nice fonts
pygame.font.init()
FONT_SMALL = pygame.font.SysFont("Segoe UI", 16)
FONT_MED = pygame.font.SysFont("Segoe UI", 20)
FONT_LARGE = pygame.font.SysFont("Segoe UI", 28, bold=True)

# ---------- DO NOT MODIFY THESE FUNCTIONS ----------
def project_3d(x, y, z):
    """Simple 3D to 2D projection"""
    px = WIDTH // 2 + int((x - y) * TILT_X / 50)
    py = HEIGHT // 2 + int((x + y) * TILT_Y / 50) - z
    return px, py

def draw_spikes(values):
    screen.fill(BG_COLOR)

    for r in range(ROWS):
        for c in range(COLS):
            v = values[r, c]

            # Clamp input between 0 and 300
            v = max(0, min(300, v))

            # Invert value so that 0 is max height, 300 is min height
            h = int((300 - v) * HEIGHT_SCALE)

            # Base position (3D coordinates)
            x, y = c * ZOOM, r * ZOOM
            base = project_3d(x, y, 0)
            top = project_3d(x, y, h)

            # Draw spike
            pygame.draw.line(screen, SPIKE_COLOR, base, top, 3)
            pygame.draw.circle(screen, (200, 200, 200), base, 2)

    pygame.display.flip()
# ---------- END OF UNCHANGED FUNCTIONS ----------

# ----- Presentation helpers (new) -----
def colormap(value, vmin=0, vmax=300):
    """Return an RGB color from cool->warm for the overlay dot."""
    t = 0 if vmax == vmin else max(0.0, min(1.0, (value - vmin) / (vmax - vmin)))
    # simple gradient: blue->cyan->green->yellow->red
    if t < 0.25:
        # blue->cyan
        k = t / 0.25
        r, g, b = 0, int(255 * k), 255
    elif t < 0.5:
        # cyan->green
        k = (t - 0.25) / 0.25
        r, g, b = 0, 255, int(255 * (1 - k))
    elif t < 0.75:
        # green->yellow
        k = (t - 0.5) / 0.25
        r, g, b = int(255 * k), 255, 0
    else:
        # yellow->red
        k = (t - 0.75) / 0.25
        r, g, b = 255, int(255 * (1 - k)), 0
    return (r, g, b)

def draw_header_bar():
    # translucent header
    header_h = 54
    surf = pygame.Surface((WIDTH, header_h), pygame.SRCALPHA)
    surf.fill((20, 20, 30, 200))
    screen.blit(surf, (0, 0))
    title = FONT_LARGE.render("Pressure Sensitive Ballet Mat", True, (230, 235, 245))
    screen.blit(title, (16, 10))

def draw_footer_help():
    footer_h = 40
    surf = pygame.Surface((WIDTH, footer_h), pygame.SRCALPHA)
    surf.fill((20, 20, 30, 180))
    screen.blit(surf, (0, HEIGHT - footer_h))
    help_text = "Controls: ←/→ tilt X | ↑/↓ tilt Y | +/- zoom | [/ ] height scale | G toggle vignette"
    txt = FONT_MED.render(help_text, True, (210, 215, 225))
    screen.blit(txt, (16, HEIGHT - footer_h + 10))

def draw_hud(fps, vmin, vmax, vavg):
    # right HUD panel
    panel_w = 240
    surf = pygame.Surface((panel_w, 180), pygame.SRCALPHA)
    surf.fill((20, 20, 30, 180))
    screen.blit(surf, (WIDTH - panel_w - 16, 64))

    lines = [
        f"FPS: {fps:>5.1f}",
        f"Serial: {'OK' if ser else 'None'}",
        f"Port: {PORT if ser else '-'}",
        f"Grid: {ROWS}×{COLS}",
        f"Zoom: {ZOOM}",
        f"TiltX/TiltY: {TILT_X}/{TILT_Y}",
        f"Height scale: {HEIGHT_SCALE:.3f}",
        f"Min/Max/Avg: {int(vmin)} / {int(vmax)} / {int(vavg)}",
    ]
    y = 80
    for line in lines:
        txt = FONT_MED.render(line, True, (230, 235, 245))
        screen.blit(txt, (WIDTH - panel_w - 4 + 16, y))
        y += 22

def draw_colorbar():
    # colorbar legend (value 0..300)
    bar_h = 220
    bar_w = 16
    x = WIDTH - 40
    y = 64
    for i in range(bar_h):
        t = 1 - i / (bar_h - 1)
        val = int(t * 300)
        pygame.draw.line(screen, colormap(val), (x, y + i), (x + bar_w, y + i))
    border = pygame.Rect(x, y, bar_w, bar_h)
    pygame.draw.rect(screen, (230, 230, 230), border, 1)

    # labels
    lbl_max = FONT_SMALL.render("300", True, (230, 230, 230))
    lbl_mid = FONT_SMALL.render("300", True, (230, 230, 230))
    lbl_min = FONT_SMALL.render("0", True, (230, 230, 230))
    screen.blit(lbl_max, (x - 30, y - 2))
    screen.blit(lbl_mid, (x - 30, y + bar_h // 2 - 8))
    screen.blit(lbl_min, (x - 30, y + bar_h - 14))
    caption = FONT_SMALL.render("Value", True, (200, 200, 210))
    screen.blit(caption, (x - 8, y + bar_h + 6))

def draw_overlay_points(values):
    """Draw colored tips and subtle glow without touching draw_spikes()."""
    vmin, vmax = np.min(values), np.max(values)
    for r in range(ROWS):
        for c in range(COLS):
            v = max(0, min(300, int(values[r, c])))
            h = int((300 - v) * HEIGHT_SCALE)
            x, y = c * ZOOM, r * ZOOM
            top = project_3d(x, y, h)

            # soft glow using translucent circles
            glow = pygame.Surface((30, 30), pygame.SRCALPHA)
            pygame.draw.circle(glow, (*colormap(v), 60), (15, 15), 12)
            pygame.draw.circle(glow, (*colormap(v), 120), (15, 15), 6)
            screen.blit(glow, (top[0] - 15, top[1] - 15))

            # crisp dot at the tip
            pygame.draw.circle(screen, colormap(v), top, 3)

def draw_axes_labels():
    # Axes arrows (screen space, purely cosmetic)
    # X arrow
    base = project_3d(0, 0, 0)
    x_end = project_3d(COLS * ZOOM * 0.9, 0, 0)
    y_end = project_3d(0, ROWS * ZOOM * 0.9, 0)
    pygame.draw.aaline(screen, (180, 180, 200), base, x_end)
    pygame.draw.aaline(screen, (180, 180, 200), base, y_end)
    # labels
    lbl_x = FONT_SMALL.render("X (columns)", True, (200, 200, 210))
    lbl_y = FONT_SMALL.render("Y (rows)", True, (200, 200, 210))
    screen.blit(lbl_x, (x_end[0] + 8, x_end[1] - 6))
    screen.blit(lbl_y, (y_end[0] + 8, y_end[1] - 6))

def draw_vignette(on=True):
    if not on:
        return
    vignette = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
    # radial alpha falloff
    center = np.array([WIDTH / 2, HEIGHT / 2])
    max_d = math.hypot(WIDTH / 2, HEIGHT / 2)
    # draw a few expanding translucent rects as a quick vignette
    for i in range(8):
        alpha = int(30 + i * 12)
        pygame.draw.rect(vignette, (0, 0, 0, alpha), (i*6, i*6, WIDTH - i*12, HEIGHT - i*12), width=6)
    screen.blit(vignette, (0, 0))

# EMA smoothing for nicer presentation
SMOOTHING = 0.35
smoothed = np.zeros((ROWS, COLS), dtype=float)

# Toggles
vignette_on = True

# ==== Main loop ====
running = True
last_time = time.time()
fps = 0.0
values = np.zeros((ROWS, COLS), dtype=int)

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            # live view tweaks (no changes to functions)
            if event.key == pygame.K_EQUALS or event.key == pygame.K_PLUS:
                ZOOM = min(60, ZOOM + 1)
            elif event.key == pygame.K_MINUS or event.key == pygame.K_UNDERSCORE:
                ZOOM = max(10, ZOOM - 1)
            elif event.key == pygame.K_LEFT:
                TILT_X = max(10, TILT_X - 2)
            elif event.key == pygame.K_RIGHT:
                TILT_X = min(80, TILT_X + 2)
            elif event.key == pygame.K_UP:
                TILT_Y = min(60, TILT_Y + 2)
            elif event.key == pygame.K_DOWN:
                TILT_Y = max(5, TILT_Y - 2)
            elif event.key == pygame.K_LEFTBRACKET:
                HEIGHT_SCALE = max(0.05, HEIGHT_SCALE - 0.01)
            elif event.key == pygame.K_RIGHTBRACKET:
                HEIGHT_SCALE = min(0.50, HEIGHT_SCALE + 0.01)
            elif event.key == pygame.K_g:
                vignette_on = not vignette_on

    # read serial
    line = None
    if ser:
        try:
            line = ser.readline().decode(errors="ignore").strip()
        except Exception:
            line = None

    if line:
        try:
            data = np.array([int(x) for x in line.split(",")])
            if len(data) == ROWS * COLS:
                raw = data.reshape((ROWS, COLS))
                # smooth
                smoothed = SMOOTHING * raw + (1 - SMOOTHING) * smoothed
                values = smoothed.astype(int)
        except Exception:
            pass

    # call your original renderer (clears + flips)
    draw_spikes(values)

    # --- overlay polish drawn AFTER spikes ---
    # we need another flip/update after adding overlays
    draw_overlay_points(values)
    draw_axes_labels()
    draw_header_bar()
    draw_footer_help()
    vmin, vmax, vavg = np.min(values), np.max(values), np.mean(values)
    # HUD and colorbar last so they sit on top
    draw_colorbar()
    # vignette for focus
    draw_vignette(vignette_on)

    # FPS
    now = time.time()
    dt = now - last_time
    last_time = now
    fps = 0.9 * fps + 0.1 * (1.0 / dt if dt > 0 else 0.0)
    draw_hud(fps, vmin, vmax, vavg)

    # update the whole screen once after overlays
    pygame.display.flip()

    clock.tick(30)

if ser:
    ser.close()
pygame.quit()
